In [32]:
!pip install gradio ultralytics google-generativeai pillow

In [33]:


import os
os.environ["GEMINI_API_KEY"] = ""#Enter your api key here
# we trained a pretrained yolov8 on similar food classification task on manually curated dataset
os.environ["YOLO_MODEL_PATH"] = "custom_model.pt" #If you store custom model file in some other folder put the same address here




In [34]:
from ultralytics import YOLO
from PIL import Image
import requests
import io
import os
import matplotlib.pyplot as plt
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
import google.generativeai as genai
import gradio as gr

In [35]:
# drive.mount('/content/drive') chamge this accordingly

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
genai.configure(api_key="")#put your gemini api key here we used the free version
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

In [40]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import google.generativeai as genai
from PIL import Image
import gradio as gr
import logging
import time
import re

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Helper function to log and print
def log_print(message, level=logging.INFO):
    print(message)
    logging.log(level, message)

# Load YOLO model
def load_yolo_model(model_path):
    log_print("Loading YOLO model")
    return YOLO(model_path)

# Perform object detection and return the first detected object
def detect_first_object(yolo_model, image_path):
    log_print("Detecting first object in the image")
    results = yolo_model.predict(source=image_path, imgsz=640, conf=0.05)
    if len(results[0].boxes) > 0:
        log_print("Object detected")
        return results[0].boxes[0]
    log_print("No objects detected in the image", level=logging.WARNING)
    return None

# Crop the first detected object
def crop_first_object(image_path, box):
    log_print("Cropping the first detected object")
    original_image = Image.open(image_path)
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped_image = original_image.crop((x1, y1, x2, y2))
    return cropped_image

# Process image and generate nutrition info
def process_image(image, height, weight, age, sex, diet, api_key, yolo_model):
    log_print("Starting image processing")

    # Save the uploaded image
    image_path = "temp_image.jpg"
    image.save(image_path)
    log_print(f"Saved uploaded image to {image_path}")

    # Detect the first object
    first_box = detect_first_object(yolo_model, image_path)

    if first_box is None:
        log_print("No food items detected in the image", level=logging.WARNING)
        return "No food items detected in the image.", "", ""

    # Crop the first object
    cropped_image = crop_first_object(image_path, first_box)
    cropped_image_path = "cropped_image.jpg"
    cropped_image.save(cropped_image_path)
    log_print("First object cropped")
    sample_file = Image.open(cropped_image_path)
    log_print("First object uploaded")
    log_print(f"Saved cropped image to {cropped_image_path}")

    # Generate content for the cropped image
    prompt = "dont give any formatting just give answer in point add linespace and nothing else give your anwer in points Estimate the quantities of all the different food items in the image. Dont add anyother words apart from what is asked Just straight out give output in cleanly formatted version."
    log_print("Initiating API call")
    nutrition_info = model.generate_content([sample_file, prompt])
    log_print("Generated nutrition information")

    # Generate macronutrient breakdown
    log_print("Generating macronutrient breakdown...")
    macro_prompt = f"dont give any formatting just give answer in point add linespace and nothing else  give your anwer in points  Using these details, give me the breakdown of macronutrients in the diet.Consider the person's height: {height}cm, weight: {weight}kg, age: {age}, and sex: {sex} diet type: {diet} Roughly estimate the quantities of food items to do your calculations. Dont add anyother words apart from what is asked Just straight out give output in cleanly formatted version."
    macro_info = model.generate_content(nutrition_info.text + macro_prompt)
    log_print("Generated macronutrient breakdown")

    # Generate RDA table
    log_print("Generating RDA table...")
    rda_prompt = f"dont give any formatting just give answer in point add linespace and nothing else  give your anwer in points  Now format this in the form of a neat and clean RDA ponts for the given individual. Consider the person's height: {height}cm, weight: {weight}kg, age: {age}, and sex: {sex} diet type: {diet}.Also suggest what his diet is lakcing what should he add suggest food items under the ehading recomendations ,Dont add anyother words apart from what is asked Just straight out give output in cleanly formatted version."
    rda_table = model.generate_content(macro_info.text + rda_prompt)
    log_print("Generated RDA table")

    return nutrition_info, macro_info, rda_table

def format_text(text):
    # Remove any markdown-style headers
    text = re.sub(r'^#+\s*', '', text, flags=re.MULTILINE)

    # Replace bullet points with proper Unicode bullets
    text = re.sub(r'^\s*[-*]\s', '• ', text, flags=re.MULTILINE)

    # Add line breaks for readability
    text = text.replace('. ', '.\n')

    # Capitalize the first letter of each sentence
    text = '. '.join(sentence.capitalize() for sentence in text.split('. '))

    return text.strip()

def format_table(table_text):
    lines = table_text.split('\n')
    formatted_lines = []
    for line in lines:
        if '|' in line:
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            formatted_line = ' | '.join(f"{cell:<20}" for cell in cells)
            formatted_lines.append(formatted_line)
        else:
            formatted_lines.append(line)
    return '\n'.join(formatted_lines)

# Gradio interface
def gradio_interface(image, height, weight, age, sex, diet):
    log_print("Starting Gradio interface function")
    api_key = os.environ.get("GEMINI_API_KEY")
    yolo_model_path = os.environ.get("YOLO_MODEL_PATH")
    yolo_model = load_yolo_model(yolo_model_path)

    nutrition_info, macro_info, rda_table = process_image(image, height, weight, age, sex, diet, api_key, yolo_model)

    # Format the outputs
    formatted_nutrition_info = format_text(nutrition_info.text)
    formatted_macro_info = format_text(macro_info.text)
    formatted_rda_table = format_table(rda_table.text)

    log_print("Final Results:")
    log_print("Nutrition Info:")
    print(formatted_nutrition_info)
    log_print("Macronutrient Breakdown:")
    print(formatted_macro_info)
    log_print("RDA Table:")
    print(formatted_rda_table)

    return formatted_nutrition_info, formatted_macro_info, formatted_rda_table

# Create Gradio interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Image(type="pil", label="Upload your meal image"),
        gr.Number(label="Height (cm)"),
        gr.Number(label="Weight (kg)"),
        gr.Number(label="Age"),
        gr.Radio(["Male", "Female"], label="Sex"),
        gr.Radio(["Vegeterian", "Non Vegeterian","Vegan","Eggiterian"], label="Diet")
    ],
    outputs=[
        gr.Textbox(label="Nutrition Information"),
        gr.Textbox(label="Macronutrient Breakdown"),
        gr.Textbox(label="Analysis and Suggestions")
    ],
    title="Diet Analysis",
    description="Upload an image of your meal to get nutrition information and RDA analysis."
)

# Launch the interface
if __name__ == "__main__":
    log_print("Launching Gradio interface")
    iface.launch(debug=True)

Launching Gradio interface
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://774215322476dd0afe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Starting Gradio interface function
Loading YOLO model
Starting image processing
Saved uploaded image to temp_image.jpg
Detecting first object in the image

image 1/1 /content/temp_image.jpg: 384x640 1 Plate, 128.9ms
Speed: 5.6ms preprocess, 128.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Object detected
Cropping the first detected object
First object cropped
First object uploaded
Saved cropped image to cropped_image.jpg
Initiating API call
Generated nutrition information
Generating macronutrient breakdown...
Generated macronutrient breakdown
Generating RDA table...
Generated RDA table
Final Results:
Nutrition Info:
• 250 ml dal makhani
• 200 ml paneer butter masala
• 150 ml malai kofta
• 100 ml mix veg
• 150 ml dal
• 200 gm rice
• 2 pieces of samosa
• 150 gm salad
• 200 gm potato
• 250 gm naan
• 50 gm onion
• 50 gm chutney
Macronutrient Breakdown:
• calories: 2400-2600 kcal
• protein: 80-90 grams
• carbohydrates: 350-400 grams
• fat: 70-80 grams
RDA Table:
- Ca